In [24]:
import pandas as pd              #analyse de donnees
import numpy as np               #manipulation de tableaux ou calcul mathematiques
import seaborn as sns            #visualisation de donnees
import matplotlib.pyplot as plt  #visualisation les données
import statsmodels.api as sm

In [25]:
from sklearn.model_selection import train_test_split
#from keras.preprocessing.image import load_img, img_to_array
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator


In [26]:
data = pd.read_csv("C:/Users/nico_/Desktop/IA School M2/Fast Fashion/visuelle2/sales.csv")

# Création d'une matière polluante

In [27]:
liste_polluant= ['acrylic','georgette', 'nylon','dark jeans','light jeans', 'heavy jeans','faux leather', 'fluid polyviscous', 'polyviscous','paillettes' ]


In [28]:
liste_ok = data['fabric'].unique().copy()

In [29]:
liste_polluant

['acrylic',
 'georgette',
 'nylon',
 'dark jeans',
 'light jeans',
 'heavy jeans',
 'faux leather',
 'fluid polyviscous',
 'polyviscous',
 'paillettes']

In [30]:
liste_non_polluant = np.setdiff1d(liste_ok, liste_polluant)

In [31]:
liste_non_polluant 

array(['angora', 'bengaline', 'cady', 'chambree', 'chanel', 'chine crepe',
       'cloth', 'cotton', 'crepe', 'dainetto', 'devore', 'embossed',
       'flamed', 'fluid', 'foam rubber', 'frise', 'fur', 'goose', 'hron',
       'ity', 'jacquard', 'lace', 'linen', 'lurex', 'macrame', 'marocain',
       'matte jersey', 'milano stitch', 'mohair', 'muslin cotton or silk',
       'mutton', 'nice', 'ottoman', 'piquet', 'plisse', 'plumetis',
       'plush', 'satin cotton', 'scottish', 'scuba crepe', 'shiny jersey',
       'silky satin', 'tactel', 'technical', 'tencel', 'tulle', 'velvet',
       'viscose twill', 'webbing'], dtype=object)

In [32]:
# Créez une colonne booléenne qui vaut True si la matière est dans la liste des matières polluantes
data['matiere_polluante'] = data['fabric'].isin(liste_polluant)

# Convertissez la colonne booléenne en une colonne d'entiers (0 pour False, 1 pour True)
data['matiere_polluante'] = data['matiere_polluante'].astype(int)

In [33]:
print(data['matiere_polluante'].value_counts())

0    75030
1    31820
Name: matiere_polluante, dtype: int64


In [34]:
count = 0
for fabric in data['fabric']:
    if fabric in liste_polluant:
        count += 1
print(count)

31820


In [35]:
df = data.drop(['Unnamed: 0','external_code', 'retail', 'season', 'category', 'color','release_date', 'restock','fabric', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11'], axis=1)

In [36]:
df['matiere_polluante'] = df['matiere_polluante'].astype(str)

In [37]:
df.head()

,image_path,matiere_polluante
0,PE17/00005.png,1
1,PE17/00002.png,1
2,PE17/00005.png,1
3,PE17/00009.png,0
4,PE17/00005.png,1


In [38]:
# extraire les noms de fichiers
new_paths = []
for path in df['image_path']:
    #new_path = path.split('/')[-1].split('.')[0]
    new_path = path.split('/')[-1]
    new_paths.append(new_path)

# remplacer la colonne image_path par les nouveaux noms de fichiers extraits
df['image_path'] = new_paths

In [39]:
df.head()

,image_path,matiere_polluante
0,00005.png,1
1,00002.png,1
2,00005.png,1
3,00009.png,0
4,00005.png,1


# Je ne prends que les 1000 premières photos

In [40]:
import os


# Définition des chemins d'accès aux images
image_dir = r'C:\Users\nico_\Desktop\IA School M2\Fast Fashion\visuelle2\images\images1000'

# Liste de tous les fichiers dans le répertoire d'images
all_files = os.listdir(image_dir)

# Liste des fichiers dans le dataframe
df_files = df['image_path'].tolist()

# Vérification des fichiers manquants
missing_files = []
for file in df_files:
    if file not in all_files:
        missing_files.append(file)
# Suppression des lignes correspondantes dans le dataframe
df = df[~df['image_path'].isin(missing_files)]


In [41]:
df.isnull().sum()

image_path           0
matiere_polluante    0
dtype: int64

In [42]:
df.duplicated().sum()

13000

In [43]:
df.drop_duplicates(inplace=True)

In [44]:
df.duplicated().sum()

0

In [45]:
df.shape

(946, 2)

In [46]:
# Définition des chemins d'accès aux images
image_dir = r'C:\Users\nico_\Desktop\IA School M2\Fast Fashion\visuelle2\images\images1000'

# Division des données en ensembles d'entraînement et de test
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Configuration du générateur d'images
datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

# Chargement des images à partir du générateur
train_generator = datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=image_dir,
    x_col="image_path",
    y_col="matiere_polluante",
    target_size=(224, 224),
    batch_size=32,
    class_mode="binary",
    shuffle=True,
    seed=42
)

validation_generator = datagen.flow_from_dataframe(
    dataframe=test_df,
    directory=image_dir,
    x_col="image_path",
    y_col="matiere_polluante",
    target_size=(224, 224),
    batch_size=32,
    class_mode="binary",
    shuffle=True,
    seed=42
)

# Chargement du modèle VGG16 sans la dernière couche fully-connected
base_model = VGG16(weights="imagenet", include_top=False)

# Ajout d'une nouvelle couche fully-connected pour la classification binaire
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(64, activation="relu")(x)
predictions = Dense(1, activation="sigmoid")(x)

# Définition du modèle final
model = Model(inputs=base_model.input, outputs=predictions)

# Compilation du modèle
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# Entraînement du modèle
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=10
)


Found 756 validated image filenames belonging to 2 classes.
Found 190 validated image filenames belonging to 2 classes.
Epoch 1/10
23/23 [==============================] - 742s 32s/step - loss: 4.4980 - accuracy: 0.6616 - val_loss: 0.6891 - val_accuracy: 0.7500
Epoch 2/10
23/23 [==============================] - 822s 35s/step - loss: 0.6016 - accuracy: 0.7666 - val_loss: 0.5948 - val_accuracy: 0.7437
Epoch 3/10
23/23 [==============================] - 780s 34s/step - loss: 0.5677 - accuracy: 0.7652 - val_loss: 0.5971 - val_accuracy: 0.7437
Epoch 4/10
23/23 [==============================] - 766s 33s/step - loss: 0.5505 - accuracy: 0.7666 - val_loss: 0.5919 - val_accuracy: 0.7500
Epoch 5/10
23/23 [==============================] - 793s 34s/step - loss: 0.5950 - accuracy: 0.7624 - val_loss: 0.6520 - val_accuracy: 0.7375
Epoch 6/10
23/23 [==============================] - 800s 35s/step - loss: 0.6190 - accuracy: 0.7500 - val_loss: 0.5738 - val_accuracy: 0.7625
Epoch 7/10
23/23 [==========

In [48]:
# Score du modèle
score = model.evaluate(validation_generator)
print("Test Loss:", score[0])
print("Test Accuracy:", score[1])

6/6 [==============================] - 58s 10s/step - loss: 0.5624 - accuracy: 0.7526
Test Loss: 0.562414288520813
Test Accuracy: 0.7526316046714783


# Test image

In [50]:
from tensorflow.keras.preprocessing import image

# Chemin d'accès à l'image à prédire
img_path = r"C:\Users\nico_\Desktop\IA School M2\Fast Fashion\visuelle2\images\vest_jean.jpg"

# Chargement de l'image
img = image.load_img(img_path, target_size=(224, 224))

# Prétraitement de l'image
img = image.img_to_array(img)
img = np.expand_dims(img, axis=0)
img = preprocess_input(img)

# Prédiction de la classe de l'image
prediction = model.predict(img)

# Affichage de la prédiction
if prediction == 0:
    print("L'image ne contient pas de matière polluante.")
else:
    print("L'image contient de la matière polluante.")

1/1 [==============================] - 4s 4s/step
L'image contient de la matière polluante.


In [51]:
from tensorflow.keras.preprocessing import image

# Chemin d'accès à l'image à prédire
img_path = r"C:\Users\nico_\Desktop\IA School M2\Fast Fashion\visuelle2\images\vest_jean.jpg"

# Chargement de l'image
img = image.load_img(img_path, target_size=(224, 224))

# Prétraitement de l'image
img = image.img_to_array(img)
img = np.expand_dims(img, axis=0)
img = preprocess_input(img)

# Prédiction de la classe de l'image
prediction = model.predict(img)

# Affichage de la prédiction
if prediction == 0:
    print("L'image ne contient pas de matière polluante.")
else:
    pourcentage = round(float(prediction[0][0]) * 100, 2)    # En utilisant l'indice [0], nous pouvons extraire ce tableau, et en utilisant l'indice [0][0], nous pouvons extraire la première et unique valeur de ce tableau.
    print(f"L'image contient de la matière polluante avec une probabilité de {pourcentage}%.")


1/1 [==============================] - 2s 2s/step
L'image contient de la matière polluante avec une probabilité de 21.75%.
